<a href="https://colab.research.google.com/github/wnttoknow/Leetcode/blob/main/leetcode_sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Info
В этом ноутбуке решенные мной задачи с leetcode на тему SQL уровня **medium и hard**.

Непосредственно в ноутбуке использовал sqlite3, в leetcode - MySQL.

Для этого немного адаптировал CRUDE запросы (SQLSchema с leetcode), также некоторые функции и особенности диалектов.


# Imports

Это апгрейд sqlite3 чтобы были оконные функции.

https://stackoverflow.com/a/59429952/4527289

В следующей ячейке в конце стоит команда os.kill -- она убьет текущий рантайм. Это нужно для того, чтобы заработала обновленная версия sqlite3. Когда запускаешь эту ячейку colab выдаст сообщение об ошибке -- так и должно быть. Просто запускаем следующую ячейку.

In [ ]:
!gdown 1CDqqyKV9BvrvnuR1l8HVz0aYuM4yOTUr
!mv _sqlite3.cpython-38-x86_64-linux-gnu.so /usr/lib/python3.8/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1CDqqyKV9BvrvnuR1l8HVz0aYuM4yOTUr
To: /content/_sqlite3.cpython-38-x86_64-linux-gnu.so
100% 6.59M/6.59M [00:00<00:00, 81.8MB/s]


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import sqlite3

In [ ]:
sqlite3.sqlite_version

'3.40.0'

In [ ]:
con = sqlite3.connect('db')
cur = con.cursor()

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
def create_table(sql):
  sql = ';\n'.join(sql.split('\n'))
  cur.executescript(sql)

# 176 Second Highest Salary
https://leetcode.com/problems/second-highest-salary/description/

Логика решения:
1. Ранжируем с помощью оконной функции dense_rank внутри нее сортируем по уменьшению зарплаты.
2. Выставляем условие ранга = 2. При селекте используем функцию max(), так как равные зарплаты имеют одинаковый ранг, а нам нужен один вывод. Также max хорошо отработает и выведет None (как требуется в условии) в случае, если второй самой высокой зарплаты нет.

In [ ]:
sql = ''' drop table if exists Employee
Create table Employee (Id int, Salary int)
insert into Employee (Id, Salary) values ('1', '100')
insert into Employee (Id, Salary) values ('2', '200')
insert into Employee (Id, Salary) values ('2', '200')
insert into Employee (Id, Salary) values ('3', '300')'''

In [ ]:
#для sqlite нужно в явном виде проставить ;
sql = ';\n'.join(sql.split('\n'))

In [ ]:
print(sql)

 drop table if exists Employee;
Create table Employee (Id int, Salary int);
insert into Employee (Id, Salary) values ('1', '100');
insert into Employee (Id, Salary) values ('2', '200');
insert into Employee (Id, Salary) values ('2', '200');
insert into Employee (Id, Salary) values ('3', '300')


In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select * from Employee t'''

In [ ]:
select(sql)

,Id,Salary
0,1,100
1,2,200
2,2,200
3,3,300


In [ ]:
# решение задачи с помощью функции ранжирования и max()
sql = '''
with ranked as  
(select dense_rank() over(order by e.salary desc) as rnk,
e.salary 
from Employee as e)

select max(ranked.salary) as SecondHighestSalary
from ranked
where ranked.rnk = 2
'''

In [ ]:
select(sql)

,SecondHighestSalary
0,200
1,200


In [ ]:
# создадим test case для проверки
# есть вопросы к правильному выбору функции ранжирования
sql = '''drop table if exists Employee_tc1;
Create table Employee_tc1 (Id int, Salary int);
insert into Employee_tc1 (Id, Salary) values ('1', '100');
insert into Employee_tc1 (Id, Salary) values ('2', '100')'''


In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''
select *
from Employee_tc1
'''

In [ ]:
select(sql)

,Id,Salary
0,1,100
1,2,100


In [ ]:
sql = '''
with ranked as  
(select dense_rank() over(order by e.salary desc) as rnk,
e.salary 
from Employee_tc1 as e)

select max(ranked.salary) as SecondHighestSalary
from ranked
where ranked.rnk = 2'''

In [ ]:
select(sql)

,SecondHighestSalary
0,None


In [ ]:
# попробуем другое решение
sql = '''
select ifnull((select distinct t.salary 
from Employee t
order by t.salary desc
limit 1 offset 1), null) as SecondHighestSalary
'''

In [ ]:
select(sql)

,SecondHighestSalary
0,200


In [ ]:
# и еще одно
# ищем максимальную зарплату среди зарплат, которые меньше максимальной из исходной таблицы
# что и будет второй самой высокой зарплатой
sql = '''
select max( t.Salary) as SecondHighestSalary
from Employee t
where Salary < (select max(Salary) as HighestSalary from Employee)
'''

In [ ]:
select(sql)

,SecondHighestSalary
0,200


# 178 Rank scores
https://leetcode.com/problems/rank-scores/

Тут нужно воспользоваться подходящей под условие задачи ранжирующей функией. В данном случае это dense_rank.

In [ ]:
sql = '''
Create table If Not Exists Scores (id int, score DECIMAL(3,2))
insert into Scores (id, score) values ('1', '3.5')
insert into Scores (id, score) values ('2', '3.65')
insert into Scores (id, score) values ('3', '4.0')
insert into Scores (id, score) values ('4', '3.85')
insert into Scores (id, score) values ('5', '4.0')
insert into Scores (id, score) values ('6', '3.65')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Scores
'''

In [ ]:
select(sql)

,id,score
0,1,3.50
1,2,3.65
2,3,4.00
3,4,3.85
4,5,4.00
5,6,3.65


In [ ]:
# тут все просто - из трех ранжирующих оконных функций используем dense_rank()
sql = '''
select t.score,
dense_rank() over (order by t.score desc) as rank 
from Scores t
'''

In [ ]:
select(sql)

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


# 181 Employees Earning More Than Their Managers
https://leetcode.com/problems/employees-earning-more-than-their-managers/description/


Логика решения:  
сначала выделяем только сотрудников, потом джоиним их с мэнеджарами по id и сравниваем зарплаты в одной таблице

In [ ]:
sql = '''drop table if exists Employee;
Create table Employee (id int, name varchar(255), salary int, managerId int);
insert into Employee (id, name, salary, managerId) values ('1', 'Joe', '70000', '3');
insert into Employee (id, name, salary, managerId) values ('2', 'Henry', '80000', '4');
insert into Employee (id, name, salary, managerId) values ('3', 'Sam', '60000', null);
insert into Employee (id, name, salary, managerId) values ('4', 'Max', '90000', null)'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''
select *
from Employee
'''

In [ ]:
select(sql)

,id,name,salary,managerId
0,1,Joe,70000,3.0
1,2,Henry,80000,4.0
2,3,Sam,60000,NaN
3,4,Max,90000,NaN


In [ ]:
# протестируем подзапрос с определением сотрудников
sql = '''
select t.name
from Employee t
where managerId is not null
'''

In [ ]:
select(sql)

,name
0,Joe
1,Henry


In [ ]:
# сначала выделяем сотрудников, потом джоиним их с мэнеджарами по id и сравниваем зарплаты
sql = '''
select e.name as Employee
from (select *
from Employee
where managerId is not null) as e
left join Employee m on e.managerId = m.id
where e.salary > m.salary
'''

In [ ]:
select(sql)

,Employee
0,Joe


In [ ]:
# в целом выделять сотрудников подзапросом не обязательно, можно сразу джоинить
# не обязательно даже пользоваться inner джоином, после условия отпадет все ненужное
sql = '''
select e.name as Employee
from employee as e
inner join employee as m on e.managerId = m.id 
where e.salary > m.salary
'''

In [ ]:
select(sql)

,Employee
0,Joe


# 185 Department Top Three Salaries
https://leetcode.com/problems/department-top-three-salaries/

Логика решениея задачи:
1. Джоиним департамент к таблице с сторудниками
2. Расставляем dense_rank(именно он подходит по условию задачи) по окну департамента с сортировкой по зарплате
3. Селектим по условию попадания ранга <= 3


In [ ]:
sql = '''
drop table if exists Employee
drop table if exists Department
Create table Employee (id int, name varchar(255), salary int, departmentId int)
Create table Department (id int, name varchar(255))
insert into Employee (id, name, salary, departmentId) values ('1', 'Joe', '85000', '1')
insert into Employee (id, name, salary, departmentId) values ('2', 'Henry', '80000', '2')
insert into Employee (id, name, salary, departmentId) values ('3', 'Sam', '60000', '2')
insert into Employee (id, name, salary, departmentId) values ('4', 'Max', '90000', '1')
insert into Employee (id, name, salary, departmentId) values ('5', 'Janet', '69000', '1')
insert into Employee (id, name, salary, departmentId) values ('6', 'Randy', '85000', '1')
insert into Employee (id, name, salary, departmentId) values ('7', 'Will', '70000', '1')
insert into Department (id, name) values ('1', 'IT')
insert into Department (id, name) values ('2', 'Sales')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Employee
'''

In [ ]:
select(sql)

,id,name,salary,departmentId
0,1,Joe,85000,1
1,2,Henry,80000,2
2,3,Sam,60000,2
3,4,Max,90000,1
4,5,Janet,69000,1
5,6,Randy,85000,1
6,7,Will,70000,1


In [ ]:
sql = '''
select * from Department
'''

In [ ]:
select(sql)

,id,name
0,1,IT
1,2,Sales


In [ ]:
sql = '''
with t as (
select e.name, e.salary, d.name as department
from Employee as e
left join Department as d on e.departmentId = d.id  
),

r as (
select t.department, t.name, t.salary,
dense_rank() over (partition by t.department order by t.salary desc) as rnk
from t
)

select r.department as Department, 
r.name as Employee,
r.salary as Salary
from r
where r.rnk in (1, 2, 3)
'''

In [ ]:
select(sql)

,Department,Employee,Salary
0,IT,Max,90000
1,IT,Joe,85000
2,IT,Randy,85000
3,IT,Will,70000
4,Sales,Henry,80000
5,Sales,Sam,60000


In [ ]:
# немного упростим синтаксис запроса и уберем один подзапрос
sql = '''
select Department, Employee, Salary
from (
    select d.name as Department,
    e.name as Employee,
    e.salary as Salary,
    dense_rank() over (partition by d.name order by e.salary desc) as rnk
    from Employee as e left join Department as d on e.departmentId = d.id
    ) as t
where t.rnk <=3
'''

In [ ]:
select(sql)

,Department,Employee,Salary
0,IT,Max,90000
1,IT,Joe,85000
2,IT,Randy,85000
3,IT,Will,70000
4,Sales,Henry,80000
5,Sales,Sam,60000


# 180 Consecutive Numbers
https://leetcode.com/problems/consecutive-numbers/

Логика решения задачи:  
Так как по условию на нужны только повторяющиеся не менее трех раз числа, то можно создать две дополнительные колонки - одну с lag другую с lead. Шаг у обоих 1.  
Таким образом, те строки где фактическое число будет равнятся и отстающему и опережающему, как раз и будут содержать повторяющееся трижды число.  
По результатам дополнительных тесткейсов добавил:  
- distinct для вывода. Так как, если будут числа, повторяющееся > 3 раз, то они будут дублироваться в выводе.
- изменил условие с t.num = t.lag = t.lead на t.num = t.lag AND t.lag = t.lead. Так как оно почему то не работает для отрицательных чисел (например -1);
- в MySQL зарезервированы имена lag и lead, поэтому использовал другие имена колонок


In [ ]:
sql = '''
drop table if exists Logs
Create table Logs (id int, num int)
insert into Logs (id, num) values ('1', '1')
insert into Logs (id, num) values ('2', '1')
insert into Logs (id, num) values ('3', '1')
insert into Logs (id, num) values ('4', '2')
insert into Logs (id, num) values ('5', '1')
insert into Logs (id, num) values ('6', '2')
insert into Logs (id, num) values ('7', '2')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Logs
'''

In [ ]:
select(sql)

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [ ]:
sql = '''
select l.*,
lag(l.num, 1) over (order by l.id) as lag,
lead(l.num, 1) over (order by l.id) as lead
from Logs l
'''

In [ ]:
select(sql)

,id,num,lag,lead
0,1,1,NaN,1.0
1,2,1,1.0,1.0
2,3,1,1.0,2.0
3,4,2,1.0,1.0
4,5,1,2.0,2.0
5,6,2,1.0,2.0
6,7,2,2.0,NaN


In [ ]:
# решение
sql = '''
select t.num as ConsecutiveNums from
(select l.*,
lag(l.num, 1) over (order by l.id) as lag,
lead(l.num, 1) over (order by l.id) as lead
from Logs l) t
where t.num = t.lag = t.lead
'''

In [ ]:
select(sql)

,ConsecutiveNums
0,1


In [ ]:
# testcase
sql = '''
drop table if exists Logs_tc
Create table Logs_tc (id int, num int)
insert into Logs_tc (id, num) values ('1', '-1')
insert into Logs_tc (id, num) values ('2', '-1')
insert into Logs_tc (id, num) values ('3', '-1')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Logs_tc
'''

In [ ]:
select(sql)

,id,num
0,1,-1
1,2,-1
2,3,-1


In [ ]:
# финальное решение
sql = '''
select distinct t.num as ConsecutiveNums from
(select l.*,
lag(l.num, 1) over (order by l.id) as lagnum,
lead(l.num, 1) over (order by l.id) as leadnum
from Logs l) t
where t.num = t.lagnum and t.lagnum = t.leadnum
'''


In [ ]:
select(sql)

,ConsecutiveNums
0,1


In [ ]:
# этот вариант я скопипастил с интересного решения на leetcode
sql = '''
select distinct Num as ConsecutiveNums
from Logs
where (Id + 1, Num) in (select * from Logs) and (Id + 2, Num) in (select * from Logs)
'''

In [ ]:
select(sql)

,ConsecutiveNums
0,1


# 262 Trips and users
https://leetcode.com/problems/trips-and-users/


Тут пришлось повозиться с запросом по созданию таблиц, так как в sqlite3 нет типа ENUM.  
Решение https://stackoverflow.com/questions/5299267/how-to-create-enum-type-in-sqlite

Логика решения с помощью подзапросов:
1. Джоиним статус banned дважды для клиента и для такси
2. Считаем количество завершенных заказов с группировкой по дню
3. Считаем количество отмененных заказов с группировкой по дню
4. Джоиним результаты подзапросов с количеством авершенных и отмененных заказов и считаем необходимую долю. Здесь возникает много вопросов с особенностями деления в SQL и NaN. Нужно пользоваться функциями CAST as float и COALESCE.
5. В sqlite нет outer join, по хорошему он нам нужен в конце, так как на разные даты может не быть отмененных или завершенных поездок и при объединении информация об этом пропадет

Логика решание без джоина (оптимальное решение):
1. Сразу подсчитываем отмененные заказы по условию case when t.status != 'completed' then true else null end. Здесь для правильного подсчета count обязательно поставить true/null а не 1/0. Либо вместо count можно использовать sum. Также знаменатель нужно явно преобразовать в вещественное число, чтобы получить дробный результат деления. Иначе 1/2 = 0 
2. Так как user_id для клиентов и таксистов не пересекаются, то проверку на незабанненость можно выполнить подзапросом client_id in (select users_id from Users where banned = 'No').
3. Добавляем условие по дате и групируем по дню.
Успех ! 

In [ ]:
sql = '''drop table if exists Trips;
drop table if exists Users;
Create table Trips (id int, client_id int, driver_id int, city_id int, status TEXT CHECK(status IN ('completed', 'cancelled_by_driver', 'cancelled_by_client')), request_at varchar(50))
Create table Users (users_id int, banned varchar(50), role TEXT CHECK(role IN ('client', 'driver', 'partner')))
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'completed', '2013-10-01')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('3', '3', '12', '6', 'completed', '2013-10-01')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('4', '4', '13', '6', 'cancelled_by_client', '2013-10-01')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('5', '1', '10', '1', 'completed', '2013-10-02')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('6', '2', '11', '6', 'completed', '2013-10-02')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('7', '3', '12', '6', 'completed', '2013-10-02')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('8', '2', '12', '12', 'completed', '2013-10-03')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('9', '3', '10', '12', 'completed', '2013-10-03')
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03')
insert into Users (users_id, banned, role) values ('1', 'No', 'client')
insert into Users (users_id, banned, role) values ('2', 'Yes', 'client')
insert into Users (users_id, banned, role) values ('3', 'No', 'client')
insert into Users (users_id, banned, role) values ('4', 'No', 'client')
insert into Users (users_id, banned, role) values ('10', 'No', 'driver')
insert into Users (users_id, banned, role) values ('11', 'No', 'driver')
insert into Users (users_id, banned, role) values ('12', 'No', 'driver')
insert into Users (users_id, banned, role) values ('13', 'No', 'driver')'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Trips
'''

In [ ]:
select(sql)

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [ ]:
sql = '''
select * from Users
'''

In [ ]:
select(sql)

,users_id,banned,role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [ ]:
# проверяем подзапрос
sql = '''
select t.*,
c.banned as client_banned,
d.banned as driver_banned
from Trips t
left join Users c on t.client_id = c.users_id 
left join Users d on t.driver_id = d.users_id
'''

In [ ]:
select(sql)

,id,client_id,driver_id,city_id,status,request_at,client_banned,driver_banned
0,1,1,10,1,completed,2013-10-01,No,No
1,2,2,11,1,cancelled_by_driver,2013-10-01,Yes,No
2,3,3,12,6,completed,2013-10-01,No,No
3,4,4,13,6,cancelled_by_client,2013-10-01,No,No
4,5,1,10,1,completed,2013-10-02,No,No
5,6,2,11,6,completed,2013-10-02,Yes,No
6,7,3,12,6,completed,2013-10-02,No,No
7,8,2,12,12,completed,2013-10-03,Yes,No
8,9,3,10,12,completed,2013-10-03,No,No
9,10,4,13,12,cancelled_by_driver,2013-10-03,No,No


In [ ]:
sql = '''
select s.request_at as Day, 
s.status,
count(s.id)
from (
    select t.*,
    c.banned as client_banned,
    d.banned as driver_banned
    from Trips t
    left join Users c on t.client_id = c.users_id 
    left join Users d on t.driver_id = d.users_id
    ) as s
where status = 'completed' and client_banned = 'No' and driver_banned = 'No'
group by s.request_at, status

'''

In [ ]:
select(sql)

,Day,status,count(s.id)
0,2013-10-01,completed,2
1,2013-10-02,completed,2
2,2013-10-03,completed,1


In [ ]:
# попробуем записать все решение
sql = '''
with joined as (
    select t.*,
    c.banned as client_banned,
    d.banned as driver_banned
    from Trips t
    left join Users c on t.client_id = c.users_id 
    left join Users d on t.driver_id = d.users_id
    where client_banned = 'No' and driver_banned = 'No'
    ),

completed as (
    select joined.request_at as completed_day, 
    joined.status,
    count(joined.id) as comleted_count
    from joined
    where status = 'completed'
    group by joined.request_at
    ),

canceled as (
    select joined.request_at as canceled_day, 
    joined.status,
    count(joined.id) as canceled_count
    from joined
    where status in ('cancelled_by_driver', 'cancelled_by_client') 
    group by joined.request_at
    )

select completed.completed_day as Day,
coalesce(canceled.canceled_count, 0),
completed.comleted_count, 
round(cast(canceled_count as real)/(cast(canceled_count as real) + cast(comleted_count as real)), 2) as Cancellation_Rate
from completed left join canceled on completed.completed_day = canceled.canceled_day  

'''

In [ ]:
select(sql)

,Day,"coalesce(canceled.canceled_count, 0)",comleted_count,Cancellation_Rate
0,2013-10-01,1,2,0.33
1,2013-10-02,0,2,NaN
2,2013-10-03,1,1,0.50


In [ ]:
# как тут работает деление
sql = '''
select round(cast(1 as real)/cast(3 as real), 2)
'''

In [ ]:
select(sql)

,"round(cast(1 as real)/cast(3 as real), 2)"
0,0.33


In [ ]:
# данное решение работает при проверке, но при сабмите на тесткейсе выдает неверное решение
sql = '''
with joined as (
    select t.*,
    c.banned as client_banned,
    d.banned as driver_banned
    from Trips t
    left join Users c on t.client_id = c.users_id 
    left join Users d on t.driver_id = d.users_id
    where client_banned = 'No' and driver_banned = 'No'
    ),

completed as (
    select joined.request_at as completed_day, 
    joined.status,
    count(joined.id) as comleted_count
    from joined
    where status = 'completed'
    group by joined.request_at
    ),

canceled as (
    select joined.request_at as canceled_day, 
    joined.status,
    count(joined.id) as canceled_count
    from joined
    where status in ('cancelled_by_driver', 'cancelled_by_client') 
    group by joined.request_at
    )

select completed.completed_day as Day,
coalesce(round(cast(canceled_count as real)/(canceled_count + comleted_count), 2), 0) as Cancellation_Rate
from completed left join canceled on completed.completed_day = canceled.canceled_day  

'''


In [ ]:
select(sql)

,Day,Cancellation_Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


In [ ]:
# test case tables
sql = '''
drop table if exists Trips;
drop table if exists Users;
Create table Trips (id int, client_id int, driver_id int, city_id int, status TEXT CHECK(status IN ('completed', 'cancelled_by_driver', 'cancelled_by_client')), request_at varchar(50))
Create table Users (users_id int, banned varchar(50), role TEXT CHECK(role IN ('client', 'driver', 'partner')))
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'cancelled_by_client', '2013-10-01')
insert into Users (users_id, banned, role) values ('1', 'No', 'client')
insert into Users (users_id, banned, role) values ('10', 'No', 'driver')
'''


In [ ]:
create_table(sql)

In [ ]:
sql = '''
select *
from trips, users
'''

In [ ]:
select(sql)

,id,client_id,driver_id,city_id,status,request_at,users_id,banned,role
0,1,1,10,1,cancelled_by_client,2013-10-01,1,No,client
1,1,1,10,1,cancelled_by_client,2013-10-01,10,No,driver


In [ ]:
# отладим решение по тесткейсу
sql = '''
with joined as (
    select t.*,
    c.banned as client_banned,
    d.banned as driver_banned
    from Trips t
    left join Users c on t.client_id = c.users_id 
    left join Users d on t.driver_id = d.users_id
    where client_banned = 'No' and driver_banned = 'No'
    ),

completed as (
    select joined.request_at as completed_day, 
    joined.status,
    count(joined.id) as comleted_count
    from joined
    where status = 'completed'
    group by joined.request_at
    ),

canceled as (
    select joined.request_at as canceled_day, 
    joined.status,
    count(joined.id) as canceled_count
    from joined
    where status in ('cancelled_by_driver', 'cancelled_by_client') 
    group by joined.request_at
    )

select * from canceled LEFT JOIN completed  on completed.completed_day = canceled.canceled_day
union all
select * from completed LEFT JOIN canceled on completed.completed_day = canceled.canceled_day



'''

In [ ]:
select(sql)

,canceled_day,status,canceled_count,completed_day,status,comleted_count
0,2013-10-01,cancelled_by_client,1,2013-10-01,completed,2.0
1,2013-10-03,cancelled_by_driver,1,2013-10-03,completed,1.0
2,2013-10-01,completed,2,2013-10-01,cancelled_by_client,1.0
3,2013-10-02,completed,2,None,None,NaN
4,2013-10-03,completed,1,2013-10-03,cancelled_by_driver,1.0


In [ ]:
# другой подход к решению 
sql = '''
select t.request_at as day,
round(count(case when t.status != 'completed' then true else null end) / cast(count(*) as real), 2) as 'Cancellation Rate'
from Trips as t
where (t.request_at between '2013-10-01' and '2013-10-03')
    and client_id in (select users_id from Users where banned = 'No')
    and driver_id in (select users_id from Users where banned = 'No')  
group by t.request_at

'''

In [ ]:
select(sql)

,day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


# 608 Tree Node
https://leetcode.com/problems/tree-node/description/

Самый простой подход для меня - с помощью двух условий: 
- если id есть в parent id, то значит узел является родителем для другого узла, следовательно это 'Inner'.
- если в parent id None или Null, то это "Корень"
- все остальное - листья

In [ ]:
sql = '''
drop table if exists Tree
Create table Tree (id int, p_id int)
insert into Tree (id, p_id) values ('1', 'None')
insert into Tree (id, p_id) values ('2', '1')
insert into Tree (id, p_id) values ('3', '1')
insert into Tree (id, p_id) values ('4', '2')
insert into Tree (id, p_id) values ('5', '2')
'''

In [ ]:
create_table(sql)

# 601 Human Traffic of Stadium
https://leetcode.com/problems/human-traffic-of-stadium/

Логика решения:
1. Используя оконные функции lag и lead добавим столбцы с посетителями для дня n - 2, n - 1, n + 1, n + 2. Для этих столбцов оставив только флаг, что посетителей было >= 100.
2. Дальше нужно корректно записать логическое выражение, отмечающее, что для дня n есть флаги в днях (n-2 и n-1) или (n-1 и n+1) или (n+1 и n+2). По сути отмечаем строку, где есть три флага подряд.  

In [ ]:
sql = '''
drop table if exists Stadium
Create table Stadium (id int, visit_date DATE NULL, people int)
insert into Stadium (id, visit_date, people) values ('1', '2017-01-01', '10')
insert into Stadium (id, visit_date, people) values ('2', '2017-01-02', '109')
insert into Stadium (id, visit_date, people) values ('3', '2017-01-03', '150')
insert into Stadium (id, visit_date, people) values ('4', '2017-01-04', '99')
insert into Stadium (id, visit_date, people) values ('5', '2017-01-05', '145')
insert into Stadium (id, visit_date, people) values ('6', '2017-01-06', '1455')
insert into Stadium (id, visit_date, people) values ('7', '2017-01-07', '199')
insert into Stadium (id, visit_date, people) values ('8', '2017-01-09', '188')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from stadium
'''

In [ ]:
select(sql)

,id,visit_date,people
0,1,2017-01-01,10
1,2,2017-01-02,109
2,3,2017-01-03,150
3,4,2017-01-04,99
4,5,2017-01-05,145
5,6,2017-01-06,1455
6,7,2017-01-07,199
7,8,2017-01-09,188


In [ ]:
# работает на проверке, но не работает на тесткейсе
sql = '''
select id, visit_date, people

from
    (select t.*,
    iif (t.people >= 100, 1, 0) as greater_100,
    iif(lag(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lag2,
    iif(lag(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lag1,
    iif(lead(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lead1,
    iif(lead(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lead2
    from Stadium t)

where case when greater_100 and (
  (greater_100_lead1 and greater_100_lead2)
  or (greater_100_lag1 and greater_100_lead1)
  or (greater_100_lag2 and greater_100_lag2)
) then 1 else 0 end

'''


In [ ]:
select(sql)

,id,visit_date,people
0,5,2017-01-05,145
1,6,2017-01-06,1455
2,7,2017-01-07,199
3,8,2017-01-09,188


In [ ]:
# тесткейс
sql = '''
drop table if exists Stadium
Create table Stadium (id int, visit_date DATE NULL, people int)
insert into Stadium (id, visit_date, people) values ('1', '2017-01-01', '10')
insert into Stadium (id, visit_date, people) values ('2', '2017-01-02', '109')
insert into Stadium (id, visit_date, people) values ('3', '2017-01-03', '150')
insert into Stadium (id, visit_date, people) values ('4', '2017-01-04', '99')
insert into Stadium (id, visit_date, people) values ('5', '2017-01-05', '145')
insert into Stadium (id, visit_date, people) values ('6', '2017-01-06', '1455')
insert into Stadium (id, visit_date, people) values ('7', '2017-01-07', '99')
insert into Stadium (id, visit_date, people) values ('8', '2017-01-08', '188')
insert into Stadium (id, visit_date, people) values ('8', '2017-01-09', '200')
'''


In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Stadium
'''

In [ ]:
select(sql)

,id,visit_date,people
0,1,2017-01-01,10
1,2,2017-01-02,109
2,3,2017-01-03,150
3,4,2017-01-04,99
4,5,2017-01-05,145
5,6,2017-01-06,1455
6,7,2017-01-07,99
7,8,2017-01-08,188
8,8,2017-01-09,200


In [ ]:
# не знаю почему условие так странно работает, переписал через if - заработало
sql = '''
select id, visit_date, people,
greater_100_lag2, greater_100_lag1, greater_100, greater_100_lead1, greater_100_lead2,
iif (
  greater_100 AND (
    (greater_100_lead1 AND greater_100_lead2) OR
    (greater_100_lag2 AND greater_100_lag1) OR
    (greater_100_lag1 AND greater_100_lead1)
  ), 1, 0
)  as flag

from
    (select t.*,
    iif(lag(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lag2,
    iif(lag(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lag1,
    iif (t.people >= 100, 1, 0) as greater_100,
    iif(lead(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lead1,
    iif(lead(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lead2
    from Stadium t)

'''


In [ ]:
select(sql)

,id,visit_date,people,greater_100_lag2,greater_100_lag1,greater_100,greater_100_lead1,greater_100_lead2,flag
0,1,2017-01-01,10,0,0,0,1,1,0
1,2,2017-01-02,109,0,0,1,1,0,0
2,3,2017-01-03,150,0,1,1,0,1,0
3,4,2017-01-04,99,1,1,0,1,1,0
4,5,2017-01-05,145,1,0,1,1,0,0
5,6,2017-01-06,1455,0,1,1,0,1,0
6,7,2017-01-07,99,1,1,0,1,1,0
7,8,2017-01-08,188,1,0,1,1,0,0
8,8,2017-01-09,200,0,1,1,0,0,0


In [ ]:
# итог
sql = '''
select id, visit_date, people

from
    (select t.*,
    iif(lag(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lag2,
    iif(lag(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lag1,
    iif (t.people >= 100, 1, 0) as greater_100,
    iif(lead(people, 1) over (order by visit_date) >= 100, 1, 0) as greater_100_lead1,
    iif(lead(people, 2) over (order by visit_date) >= 100, 1, 0) as greater_100_lead2
    from Stadium t)

where iif (
  greater_100 AND (
    (greater_100_lead1 AND greater_100_lead2) OR
    (greater_100_lag2 AND greater_100_lag1) OR
    (greater_100_lag1 AND greater_100_lead1)
  ), 1, 0)

'''


In [ ]:
select(sql)

,id,visit_date,people
0,5,2017-01-05,145
1,6,2017-01-06,1455
2,7,2017-01-07,199
3,8,2017-01-09,188


# 626 Exchange Seats
https://leetcode.com/problems/exchange-seats/

Логика решения:
1. Воспользуемся моими любимыми функциями lag и lead для столбца имен.
2. Зададим условие для четных id: если четное - меняем имя на предыдущее (lag), если нечетное - меняем имя на следующее (lead). Четное id определяется если остаток его деления на 2 = 0 (в MySQL есть функция mode, но я воспользовался %: id % 2 = 0).
3. На это условие сверху вешаем функцию coalesce которая в случае None будет оставлять текущее имя. Такое случается для последней строки, если количество строк нечетное.

In [ ]:
sql = '''
drop table if exists Seat
Create table Seat (id int, student varchar(255))
insert into Seat (id, student) values ('1', 'Abbot')
insert into Seat (id, student) values ('2', 'Doris')
insert into Seat (id, student) values ('3', 'Emerson')
insert into Seat (id, student) values ('4', 'Green')
insert into Seat (id, student) values ('5', 'Jeames')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from seat
'''

In [ ]:
select(sql)

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


In [ ]:
sql = '''
select t.*,
lag(t.student) over (order by t.id) as student_lag,
lead(t.student) over (order by t.id) as student_lead
from seat as t
''' 


In [ ]:
select(sql)

,id,student,student_lag,student_lead
0,1,Abbot,None,Doris
1,2,Doris,Abbot,Emerson
2,3,Emerson,Doris,Green
3,4,Green,Emerson,Jeames
4,5,Jeames,Green,None


In [ ]:
sql = '''
with s as ( 
select t.*,
lag(t.student) over (order by t.id) as student_lag,
lead(t.student) over (order by t.id) as student_lead
from seat t)

select s.id,
coalesce(case when s.id % 2 = 0 then s.student_lag
else s.student_lead end, s.student) as student 
from s
''' 


In [ ]:
select(sql)

,id,student
0,1,Doris
1,2,Abbot
2,3,Green
3,4,Emerson
4,5,Jeames


In [ ]:
sql = '''
select * from tree
'''

In [ ]:
select(sql)

,id,p_id
0,1,None
1,2,1
2,3,1
3,4,2
4,5,2


In [ ]:
sql = '''
select t.id,
case when t.p_id = 'None' then 'Root' when t.id in (select p_id from tree) then 'Inner' else 'Leaf' end as type
from tree as t
'''

In [ ]:
select(sql)

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


# 1158 Market Analysis I
https://leetcode.com/problems/market-analysis-i/

Тут самое главное правильно сджинить таблицы orders и users. И сгрупировать данные по user_id.  
Таблица Items по факту нам не требуется

In [ ]:
sql = '''
drop table if exists Users
drop table if exists Orders
drop table if exists Items
Create table Users (user_id int, join_date date, favorite_brand varchar(10))
Create table Orders (order_id int, order_date date, item_id int, buyer_id int, seller_id int)
Create table Items (item_id int, item_brand varchar(10))
insert into Users (user_id, join_date, favorite_brand) values ('1', '2018-01-01', 'Lenovo')
insert into Users (user_id, join_date, favorite_brand) values ('2', '2018-02-09', 'Samsung')
insert into Users (user_id, join_date, favorite_brand) values ('3', '2018-01-19', 'LG')
insert into Users (user_id, join_date, favorite_brand) values ('4', '2018-05-21', 'HP')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('1', '2019-08-01', '4', '1', '2')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('2', '2018-08-02', '2', '1', '3')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('3', '2019-08-03', '3', '2', '3')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('4', '2018-08-04', '1', '4', '2')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('5', '2018-08-04', '1', '3', '4')
insert into Orders (order_id, order_date, item_id, buyer_id, seller_id) values ('6', '2019-08-05', '2', '2', '4')
insert into Items (item_id, item_brand) values ('1', 'Samsung')
insert into Items (item_id, item_brand) values ('2', 'Lenovo')
insert into Items (item_id, item_brand) values ('3', 'LG')
insert into Items (item_id, item_brand) values ('4', 'HP')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Orders 
'''

In [ ]:
select(sql)

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
1,2,2018-08-02,2,1,3
2,3,2019-08-03,3,2,3
3,4,2018-08-04,1,4,2
4,5,2018-08-04,1,3,4
5,6,2019-08-05,2,2,4


In [ ]:
sql = '''
select * from users 
'''

In [ ]:
select(sql)

,user_id,join_date,favorite_brand
0,1,2018-01-01,Lenovo
1,2,2018-02-09,Samsung
2,3,2018-01-19,LG
3,4,2018-05-21,HP


In [ ]:
sql = '''
select * from items 
'''

In [ ]:
select(sql)

,item_id,item_brand
0,1,Samsung
1,2,Lenovo
2,3,LG
3,4,HP


In [ ]:
select(sql)

,user_id,join_date,favorite_brand,order_id,order_date,item_id,buyer_id,seller_id,count(o.order_id)
0,1,2018-01-01,Lenovo,1,2019-08-01,4,1,2,1
1,2,2018-02-09,Samsung,3,2019-08-03,3,2,3,2


In [ ]:
sql = '''
select u.user_id as buyer_id,
u.join_date,
coalesce(o.orders_in_2019, 0) as orders_in_2019
from Users u left join
    (select t.buyer_id,
    count(t.order_id) as orders_in_2019 
    from Orders t
    where t.order_date between '2019-01-01' and '2019-12-31'
    group by t.buyer_id) 
as o on u.user_id = o.buyer_id
'''

In [ ]:
select(sql)

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


# 1393 Capital Gain/Loss
https://leetcode.com/problems/capital-gainloss/description/

Задача изначально показалась довольно сложной и первой мыслью было для каждой компании найти пары продажа-покупка, для них вычислить прибыть и затем сложить результаты этих пар.  
По итогу оказалось что искать и соблюдать эти пары не обязательно!  
Если для одной компании записать итоговую формулу:
$$
gain/loss = \sum_{1}^{n}(sell\_price_n - buy\_price_n)
$$
, где n - кол-во пар покапка/продажа

То с учетом ассоциативности сложения станет понятно, что соблюдать пары не обязателено, важен лиш знак для цены.



In [ ]:
sql = '''
drop table if exists Stocks
Create Table Stocks (stock_name varchar(15), operation TEXT CHECK (operation IN ('Sell', 'Buy')), operation_day int, price int)
insert into Stocks (stock_name, operation, operation_day, price) values ('Leetcode', 'Buy', '1', '1000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Buy', '2', '10')
insert into Stocks (stock_name, operation, operation_day, price) values ('Leetcode', 'Sell', '5', '9000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Handbags', 'Buy', '17', '30000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Sell', '3', '1010')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Buy', '4', '1000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Sell', '5', '500')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Buy', '6', '1000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Handbags', 'Sell', '29', '7000')
insert into Stocks (stock_name, operation, operation_day, price) values ('Corona Masks', 'Sell', '10', '10000')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''
select * from Stocks
'''

In [ ]:
select(sql)

,stock_name,operation,operation_day,price
0,Leetcode,Buy,1,1000
1,Corona Masks,Buy,2,10
2,Leetcode,Sell,5,9000
3,Handbags,Buy,17,30000
4,Corona Masks,Sell,3,1010
5,Corona Masks,Buy,4,1000
6,Corona Masks,Sell,5,500
7,Corona Masks,Buy,6,1000
8,Handbags,Sell,29,7000
9,Corona Masks,Sell,10,10000


In [ ]:
sql = '''
select stock_name, operation_day,  price as buy_price
from Stocks t
where operation = 'Buy'
order by stock_name, operation_day
'''

In [ ]:
select(sql)

,stock_name,operation_day,buy_price
0,Corona Masks,2,10
1,Corona Masks,4,1000
2,Corona Masks,6,1000
3,Handbags,17,30000
4,Leetcode,1,1000


In [ ]:
sql = '''
select stock_name, 
sum(iif(operation = 'Sell', 1, -1) * price) as capital_gain_loss
from Stocks t
group by stock_name
'''

In [ ]:
select(sql)

,stock_name,capital_gain_loss
0,Corona Masks,9500
1,Handbags,-23000
2,Leetcode,8000


# 182, 196 Dealing with duplicates
https://leetcode.com/problems/delete-duplicate-emails/description/
https://leetcode.com/problems/duplicate-emails/


Это задачи легкого уровня, но очень распространенная задача обработки дубликатов

1. Дубликаты можно выявлять с помощью group by, count и условии в having count > 1
2. Непосредственную строчку с дубликатом можно выявить с помощью group by и функции min например по id. Соответственно получаем список с id строк без дубликатов и первыми встечающимися дубликатами. Все остальное можно удалить с помощью DELETE. Тут есть особенности - почему то получилось удалить только когда добавил еще один уровень в подзапрос.

In [ ]:
sql = '''
drop table if exists Person 
Create table Person (id int, email varchar(255))
insert into Person (id, email) values ('1', 'a@b.com')
insert into Person (id, email) values ('2', 'c@d.com')
insert into Person (id, email) values ('3', 'a@b.com')
'''

In [ ]:
create_table(sql)

In [ ]:
sql = '''select * from person'''

In [ ]:
select(sql)

,id,email
0,1,a@b.com
1,2,c@d.com


In [ ]:
# reportin duplicated email names
sql = '''
select t.email 
from Person t
group by t.email
having count(t.email) >= 2
'''

In [ ]:
select(sql)

,email
0,a@b.com


In [ ]:
# поиск и удаление (select заменить на delete) конкретной строки с дубликатом
sql = '''
select * from Person t
where t.id not in
(select min(id) as min_id
from  Person
group by email)
'''

In [ ]:
select(sql)

,id,email
0,3,a@b.com


In [ ]:
# удаление дубликатов
sql = '''
delete from Person  
where id not in
(select * from (select min(id) as min_id
from  Person
group by email) as temp)
'''

In [ ]:
con.executescript(sql)

In [ ]:
sql = '''select * from Person'''
select(sql)

,id,email
0,1,a@b.com
1,2,c@d.com


In [ ]:
nums = [4, 6, 7, 7]

In [ ]:
fun = []
for i in len:
  fun.append(n*2)


fun

[8, 12, 14, 14]

In [ ]:
range(2, len(nums))

range(2, 4)

In [ ]:
for _ in range(2, len(nums)+1):
  print(_)


2
3
4
